In [1]:
import sxtwl

In [2]:
lunar_date = sxtwl.fromLunar(1992, 8, 25, False)
print(lunar_date)
print(type(lunar_date))

<sxtwl.Day; proxy of <Swig Object of type 'Day *' at 0x7fe9eaf19740> >
<class 'sxtwl.Day'>


In [22]:
# 从公历年月日获取一天的信息
# day = sxtwl.fromSolar(1992, 8, 25)
day = sxtwl.fromSolar(2025, 3, 3)
print(f"公历：{day.getSolarYear()}年{day.getSolarMonth()}月{day.getSolarDay()}日", type(day))

jd = day.getJieQiJD()
jd

公历：2025年3月3日 <class 'sxtwl.Day'>


2451545.0

In [21]:
day2 = sxtwl.fromSolar(2025, 3, 4)
print(f"公历：{day2.getSolarYear()}年{day2.getSolarMonth()}月{day2.getSolarDay()}日", type(day2))

jd2 = day2.getJieQiJD()
jd2

公历：2025年3月4日 <class 'sxtwl.Day'>


2451545.0

In [29]:
# day3 = day2.after(1)
day3 = sxtwl.fromSolar(2025, 3, 5)
print(f"公历：{day3.getSolarYear()}年{day3.getSolarMonth()}月{day3.getSolarDay()}日")

jd3 = day3.getJieQiJD()
jd3

公历：2025年3月5日


2460740.1715546497

In [30]:
# day3 = day2.after(1)
day4 = sxtwl.fromSolar(2026, 3, 6)
print(f"公历：{day4.getSolarYear()}年{day4.getSolarMonth()}月{day4.getSolarDay()}日")

jd4 = day4.getJieQiJD()
jd4

公历：2026年3月6日


2451545.0

In [35]:
# day3 = day2.after(1)
day5 = sxtwl.fromSolar(2026, 3, 8)
print(f"公历：{day5.getSolarYear()}年{day5.getSolarMonth()}月{day5.getSolarDay()}日")

jd5 = day5.getJieQiJD()
jd5

公历：2026年3月8日


2451545.0

In [34]:
print(jd2, jd, jd2-jd)
print(jd3, jd2, jd3-jd2)
print(jd4, jd3, jd4-jd3)
print(jd5, jd4, jd5-jd4)

2451545.0 2451545.0 0.0
2460740.1715546497 2451545.0 9195.171554649714
2451545.0 2460740.1715546497 -9195.171554649714
2451545.0 2451545.0 0.0


In [3]:
jqmc = ["冬至", "小寒", "大寒", "立春", "雨水", "惊蛰", "春分", "清明", "谷雨", "立夏",
        "小满", "芒种", "夏至", "小暑", "大暑", "立秋", "处暑","白露", "秋分", "寒露", 
        "霜降",  "立冬", "小雪", "大雪"]

In [31]:
import copy


_day = sxtwl.fromSolar(2025, 3, 4)
_last_day = copy(_day)
cnt = 0
while True:
    # 这里可以使用after或者before，不用担心速度，这里的计算在底层仅仅是+1这么简单
    _day = _day.after(1)
    # hasJieQi的接口比getJieQiJD速度要快，你也可以使用getJieQiJD来判断是否有节气。
    if _day.hasJieQi():
        print('\n节气：%s'% jqmc[_day.getJieQi()])

        #获取节气的儒略日数， 如果说你要计算什么时间的相距多少，直接比对儒略日要方便，相信我。

        print("day:", type(_day))
        print("day:", "公历:%d年%d月%d日" % (_day.getSolarYear(), _day.getSolarMonth(), _day.getSolarDay()))

        jd = _day.getJieQiJD()
        print("jd:", jd, type(jd))

    
        # 将儒略日数转换成年月日时秒
        t = sxtwl.JD2DD(jd)
        
        # 注意，t.s是小数，需要四舍五入
        print("节气时间:%d-%d-%d %d:%d:%d"%(t.Y, t.M, t.D, t.h, t.m, round(t.s)))

        
        jd_last = _last_day.getJieQiJD()
        tt = sxtwl.JD2DD(jd_last)
        print("last day:", "公历:%d年%d月%d日" % (tt.Y, tt.M, tt.D))

        
        print("相差:", jd - jd_last)
        
        
        
        cnt += 1
        if cnt > 2:
            break


节气：惊蛰
day: <class 'sxtwl.Day'>
day: 公历:2025年3月5日
jd: 2460740.171554645 <class 'float'>
节气时间:2025-3-5 16:7:2
last day: 公历:-4712年1月1日
相差: 2460740.171554645

节气：春分
day: <class 'sxtwl.Day'>
day: 公历:2025年3月20日
jd: 2460755.2091843286 <class 'float'>
节气时间:2025-3-20 17:1:14
last day: 公历:-4712年1月1日
相差: 2460755.2091843286

节气：清明
day: <class 'sxtwl.Day'>
day: 公历:2025年4月4日
jd: 2460770.366906484 <class 'float'>
节气时间:2025-4-4 20:48:21
last day: 公历:-4712年1月1日
相差: 2460770.366906484


In [20]:
# input: someday (Y, M, D) [sxtwl.Day]
# output: 该日距离下一个节气的小时数
# algorithm:
# 1. 获取该日距离下一个节气的儒略日数
# 2. 计算该日距离下一个节气的儒略日数与该日儒略日数的差值
# 3. 将差值转换为小时数
# 4. 返回小时数
from typing import Tuple

def get_nearest_jieqi_time(Y, M, D, reverse=False) -> Tuple[str, Tuple[int, int, int, float, float, float]]:
    """
    获取该日距离下一个节气的小时数,

    注意: 传入的当天不会检查是否是节气, 如果需要检查, 请先调用hasJieQi()方法

    @param Y: 年, M: 月, D: 日
    @param reverse: 是否逆向查找

    @return: 节气名称, 节气时间(年, 月, 日, 时, 分, 秒)
    """

    day = sxtwl.fromSolar(Y, M, D)
    while True:
        if reverse:
            day = day.before(1)
        else:
            day = day.after(1)
        if day.hasJieQi():
            jieqi_name = jqmc[day.getJieQi()]
            print(jieqi_name, day.getJieQi())
            print(f"公历:{day.getSolarYear()}年{day.getSolarMonth()}月{day.getSolarDay()}日")

            # 把儒略日数转换为年月日时秒
            jd = day.getJieQiJD()
            t = sxtwl.JD2DD(jd)
            print(f"节气时间:%d-%d-%d %d:%d:%d"%(t.Y, t.M, t.D, t.h, t.m, round(t.s)))
            return jieqi_name, (t.Y, t.M, t.D, t.h, t.m, round(t.s))

    return None, None
    

jieqi_name, (Y, M, D, h, m, s) = get_nearest_jieqi_time(1998, 1, 15, reverse=True)
print(jieqi_name, Y, M, D, h, m, s)
print(f"下一个节气: {jieqi_name} {Y}-{M}-{D} {h} : {m} : {s}")

小寒 1
公历:1998年1月5日
节气时间:1998-1-5 21:18:9
小寒 1998 1 5 21.0 18.0 9
下一个节气: 小寒 1998-1-5 21.0 : 18.0 : 9


In [22]:
from datetime import datetime

# 计算起运时间（从出生时刻到）
def calculate_delta_time(time1: str, time2: str) -> dict:
    """计算两个日期之间的时间差并转换为年/月/日格式
    
    Args:
        time1: 第一个日期字符串，格式为 "YYYY-MM-DD HH:MM"
        time2: 第二个日期字符串，格式为 "YYYY-MM-DD HH:MM"
        
    Return:
        包含以下内容的字典:
        - raw_minutes: 两个日期之间的总分钟数
        - dhm: 天/小时/分钟的分解
        - ymds: 年/月/日的分解(使用3天=1年的转换规则)
    """
    format_str = "%Y-%m-%d %H:%M"
    
    # Parse datetime objects
    dt1 = datetime.strptime(time1, format_str)
    dt2 = datetime.strptime(time2, format_str)

    # Calculate minutes between dates
    delta_minutes = abs(int((dt2 - dt1).total_seconds() / 60))

    # Convert to days, hours, minutes
    days = delta_minutes // (24 * 60)
    remaining_minutes = delta_minutes % (24 * 60)
    hours = remaining_minutes // 60
    minutes = remaining_minutes % 60

    # Convert to total days including fractional
    total_days = days + (hours / 24) + (minutes / (24 * 60))

    # Convert to years/months/days using 3 days = 1 year
    years = int(total_days // 3)
    remaining_days = total_days % 3
    months = int(remaining_days * 4)
    remaining_months_in_days = (remaining_days * 4) % 1
    final_days = int(remaining_months_in_days * 30)

    return {
        "raw_minutes": delta_minutes,
        "dhm": {
            "days": days,
            "hours": hours,
            "minutes": minutes
        },
        "ymds": {
            "years": years,
            "months": months, 
            "days": final_days
        }
    }



<class 'int'> <class 'int'> <class 'int'> <class 'float'> <class 'float'> <class 'int'>
21 18
1998-01-15 6:00 1998-1-5 21:18
时间间隔：13482 分钟
时间间隔：9天 8小时 42分钟
总天数：9.36天
换算后：3年 1个月 13天


In [48]:
import copy

# 从公历年月日获取一天的信息
# day = sxtwl.fromSolar(2025, 3, 4)
day = sxtwl.fromSolar(1992, 8, 25)

# 记录节气信息
jieqi_data = []
cnt = 0

jieqi_data.append({
    'name': '生日',
    'solar_date': day,
    'solar_date_str': f"公历:{day.getSolarYear()}年{day.getSolarMonth()}月{day.getSolarDay()}日"
})

while True:
    day = day.after(1)
    if day.hasJieQi():
        # 获取节气信息
        jieqi_name = jqmc[day.getJieQi()]
        solar_date = f"公历:{day.getSolarYear()}年{day.getSolarMonth()}月{day.getSolarDay()}日"
       
        
        # 保存信息
        jieqi_data.append({
            'name': jieqi_name,
            'solar_date': copy(day),
            'solar_date_str': solar_date, 
        })
        
        cnt += 1
        if cnt >= 3:
            break

# 打印结果            
for i, data in enumerate(jieqi_data):
    print(f"\n节气: {data['name']}")
    print(f"日期: {data['solar_date_str']}")
    
    if i > 0:
        # 计算与上一个日期的差距
        curr_date = data['solar_date']
        prev_date = jieqi_data[i-1]['solar_date']
        
        # 计算相差的天数
            
        print(f"距离上一个日期相差: {curr_date.getSolarDay() - prev_date.getSolarDay()}天")



节气: 生日
日期: <sxtwl.Day; proxy of <Swig Object of type 'Day *' at 0x7f296c0b64f0> >

节气: 白露
日期: 公历:1992年9月7日


AttributeError: 'str' object has no attribute 'getSolarYear'

In [49]:

sxtwl.fromSolar(2025, 3, 4).


TypeError: in method 'toJD', argument 1 of type 'Time &'

In [50]:
import sxtwl
from datetime import datetime, timezone

def get_nearest_solar_term(year, month, day, hour, next_term=True):
    lunar = sxtwl.Lunar()  # 实例化天文历
    dt = datetime(year, month, day, hour, tzinfo=timezone.utc)
    timestamp_given = dt.timestamp()  # 转换为时间戳（秒）

    # 获取该年、前一年、下一年的节气时间
    solar_terms = []
    for y in [year - 1, year, year + 1]:
        jieqi_times = lunar.getJieQiTime(y)
        for idx, ts in enumerate(jieqi_times):
            if ts > 0:  # 有效节气时间
                solar_terms.append((ts, sxtwl.JIEQI[idx]))  # (时间戳, 节气名称)

    # 按时间排序
    solar_terms.sort()

    # 找到最近的节气
    if next_term:
        nearest_term = next((t for t in solar_terms if t[0] > timestamp_given), solar_terms[0])
    else:
        nearest_term = next((t for t in reversed(solar_terms) if t[0] < timestamp_given), solar_terms[-1])

    # 计算时间差（分钟）
    time_difference = (nearest_term[0] - timestamp_given) / 60
    return nearest_term[1], int(time_difference)

# 示例：计算 2025年3月4日 12:00 距离下一个节气的时间
print(get_nearest_solar_term(2025, 3, 4, 12, next_term=True))


AttributeError: module 'sxtwl' has no attribute 'Lunar'